In [ ]:
!pip install -q transformers accelerate bitsandbytes lexicalrichness

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
from lexicalrichness import LexicalRichness
import numpy as np
import pandas as pd
from collections import Counter

In [ ]:
model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
model_name = "Qwen/Qwen2.5-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
dataset = load_dataset("trivia_qa", "unfiltered.nocontext", split="validation[:100]")
questions = [item['question'] for item in dataset]
answers = [item['answer']['value'] for item in dataset]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

In [ ]:
def generate_responses(question, num_samples=5):
    responses = []
    for _ in range(num_samples):
        prompt = f"Answer this question briefly: {question}\nAnswer:"
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = response.split("Answer:")[-1].strip()
        responses.append(response)

    return responses

In [ ]:
all_responses = []
for i, q in enumerate(questions):
    print(f"Processing {i+1}/100: {q[:50]}...")
    responses = generate_responses(q)
    all_responses.append(responses)

Processing 1/100: Who was the man behind The Chipmunks?...
Processing 2/100: What star sign is Jamie Lee Curtis?...
Processing 3/100: Which Lloyd Webber musical premiered in the US on ...
Processing 4/100: Who was the next British Prime Minister after Arth...
Processing 5/100: Who had a 70s No 1 hit with Kiss You All Over?...
Processing 6/100: What claimed the life of singer Kathleen Ferrier?...
Processing 7/100: Rita Coolidge sang the title song for which Bond f...
Processing 8/100: To the nearest million what is the population of A...
Processing 9/100: What was the last US state to reintroduce alcohol ...
Processing 10/100: Which actress was voted Miss Greenwich Village in ...
Processing 11/100: What is the Japanese share index called?...
Processing 12/100: What was the name of Michael Jackson's autobiograp...
Processing 13/100: In which decade did stereo records first go on sal...
Processing 14/100: What was golfing great Ben Hogan's famous reply wh...
Processing 15/100: In what yea

In [ ]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

def semantic_entropy(responses):
    if len(responses) < 2:
        return 0.0

    embeddings = embedding_model.encode(responses)
    clustering = DBSCAN(eps=0.5, min_samples=1, metric='cosine').fit(embeddings)
    labels = clustering.labels_

    cluster_counts = Counter(labels)
    total = len(labels)
    probs = [count / total for count in cluster_counts.values()]

    entropy = -sum(p * np.log(p) for p in probs if p > 0)
    return entropy

def self_consistency(responses):
    if len(responses) < 2:
        return 1.0

    normalized = [r.strip().lower()[:50] for r in responses]
    most_common = Counter(normalized).most_common(1)[0][1]
    consistency = most_common / len(responses)

    return 1 - consistency

def token_entropy_approx(responses):
    all_tokens = []
    for response in responses:
        tokens = response.lower().split()
        all_tokens.extend(tokens)

    if len(all_tokens) == 0:
        return 0.0

    token_counts = Counter(all_tokens)
    total = len(all_tokens)
    probs = [count / total for count in token_counts.values()]

    entropy = -sum(p * np.log(p) for p in probs if p > 0)
    return entropy

def perplexity_approx(responses):
    token_ent = token_entropy_approx(responses)
    return np.exp(token_ent)

def lexical_diversity(responses):
    combined_text = " ".join(responses)

    if len(combined_text.strip()) == 0:
        return 0.0

    lex = LexicalRichness(combined_text)
    try:
        ttr = lex.ttr
    except:
        ttr = 0.0

    return ttr

In [ ]:
results = []

for i, responses in enumerate(all_responses):
    se = semantic_entropy(responses)
    sc = self_consistency(responses)
    te = token_entropy_approx(responses)
    ppl = perplexity_approx(responses)
    ld = lexical_diversity(responses)

    results.append({
        'question': questions[i],
        'semantic_entropy': se,
        'self_consistency': sc,
        'token_entropy': te,
        'perplexity': ppl,
        'lexical_diversity': ld
    })

df = pd.DataFrame(results)

In [ ]:
def check_correctness(response, ground_truth):
    response_lower = response.strip().lower()
    truth_lower = ground_truth.strip().lower()

    if truth_lower in response_lower:
        return True

    response_words = set(response_lower.split())
    truth_words = set(truth_lower.split())
    overlap = len(response_words & truth_words) / max(len(truth_words), 1)

    return overlap > 0.5

correctness = []
for i, responses in enumerate(all_responses):
    correct = any(check_correctness(r, answers[i]) for r in responses)
    correctness.append(correct)

df['correct'] = correctness
print(f"Accuracy: {sum(correctness)/len(correctness):.2%}")

Accuracy: 35.00%


In [ ]:
def calculate_ece(predictions, correctness, n_bins=10):
    df_temp = pd.DataFrame({
        'pred': predictions,
        'correct': correctness
    })

    df_temp['bin'] = pd.cut(df_temp['pred'], bins=n_bins, labels=False, duplicates='drop')

    ece = 0.0
    for bin_idx in df_temp['bin'].unique():
        if pd.isna(bin_idx):
            continue

        bin_data = df_temp[df_temp['bin'] == bin_idx]

        if len(bin_data) == 0:
            continue

        avg_confidence = bin_data['pred'].mean()
        avg_accuracy = bin_data['correct'].mean()
        bin_weight = len(bin_data) / len(df_temp)

        ece += bin_weight * abs(avg_confidence - avg_accuracy)

    return ece

metric_names = ['semantic_entropy', 'self_consistency', 'token_entropy', 'perplexity', 'lexical_diversity']

print("\nTriviaQA (Factual Category) ECE Results:")
for metric in metric_names:
    normalized = (df[metric] - df[metric].min()) / (df[metric].max() - df[metric].min() + 1e-8)
    ece = calculate_ece(normalized, df['correct'])
    print(f"{metric}: ECE = {ece:.4f}")


TriviaQA (Factual Category) ECE Results:
semantic_entropy: ECE = 0.3727
self_consistency: ECE = 0.4250
token_entropy: ECE = 0.3804
perplexity: ECE = 0.2076
lexical_diversity: ECE = 0.2091


In [ ]:
df.to_csv('triviaqa_results.csv', index=False)